In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from scipy.stats import trim_mean

from functools import partial

import altair as alt
from vega_datasets import data

# Analyse influence of weather data

## All berlin data

#### You can get the data from here:
On the following link you will find a list of zip files, one for each station with available data. For this analysis I chose the station in Berlin Tempelhof, which has the ID 433.

https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/

There is also a description file, from which I'll paste the most important content, the columns description, here:

- FX daily maximum of windgust m/s 
- FM daily mean of wind velocity m/s 
- QN_4 quality level of next columns coding see paragraph "Quality information" 
- RSK daily precipitation height mm 
- RSKF precipitation form
  - 0 no precipitation(conventional or automaticmeasurement), relates to WMO code 100
  - 1 only rain (before 1979)
  - 4 unknown form of recorded precipitation
  - 6 only rain; only liquid precipitation at automaticstations, relates to WMO code 11
  - 7 only snow; only solidprecipitation at automaticstations, relates to WMO code 12
  - 8 rain and snow (and/or"Schneeregen"); liquidand solid precipitation at automatic stations, relatesto WMO code 13 
  - 9 error or missingvalue or no automatic determination of precipitation form, relatesto WMO code 15
- SDK daily sunshine duration h
- SHK_TAG daily snow depth cm
- NM daily mean of cloud cover 1/8
- VPM daily mean of vaporpressure hPa
- PM daily mean of pressure hPa 
- TMK daily mean of temperature °C 
- UPM daily mean of relative humidity % 
- TXK daily maximum of temperature at 2m height °C 
- TNK daily minimum of temperature at 2m height °C
- TGK daily minimum of air temperature at 5cm above ground °C 
- eor End of data record

In [2]:
df_berlin = pd.read_table('data/produkt_klima_tag_20181018_20200419_00433.txt', delimiter=';')

In [3]:
df_berlin = pd.read_table('data/produkt_klima_tag_20181122_20200524_00433.txt', delimiter=';')

In [4]:
df_berlin_historic = pd.read_table('data/produkt_klima_tag_19480101_20181231_00433.txt', delimiter=';')

In [5]:
df_berlin.shape

(550, 19)

In [6]:
df_berlin_historic.shape

(25933, 19)

In [7]:
df_berlin_historic.head()

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
0,433,19480101,-999,-999.0,-999.0,5,5.7,8,-999.0,4,5.0,4.9,1010.7,-1.8,89.0,0.8,-5.0,-4.7,eor
1,433,19480102,-999,-999.0,-999.0,5,3.3,1,-999.0,8,8.0,7.9,999.7,4.4,95.0,6.8,-2.7,-2.5,eor
2,433,19480103,-999,-999.0,-999.0,5,4.0,1,-999.0,0,5.3,9.8,1001.7,8.5,86.0,10.6,5.2,4.6,eor
3,433,19480104,-999,-999.0,-999.0,5,0.2,1,-999.0,0,8.0,10.2,999.8,10.2,83.0,11.5,6.6,4.7,eor
4,433,19480105,-999,-999.0,-999.0,5,2.0,1,-999.0,0,7.7,9.8,987.8,10.5,78.0,12.0,8.0,5.9,eor


In [8]:
df_berlin = pd.concat([df_berlin, df_berlin_historic], axis=0, sort=False)

In [9]:
df_berlin.shape

(26483, 19)

In [10]:
df_berlin['time'] = pd.to_datetime(df_berlin['MESS_DATUM'], format='%Y%m%d')
df_berlin.head()

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor,time
0,433,20181122,10,8.8,4.3,3,0.0,0,0.0,-999,-999.0,5.3,1018.83,1.1,80.79,1.8,0.6,0.3,eor,2018-11-22
1,433,20181123,10,5.8,2.7,3,0.0,4,0.0,-999,-999.0,6.4,1016.12,2.0,90.29,3.4,0.5,0.2,eor,2018-11-23
2,433,20181124,10,4.1,2.1,3,0.1,4,0.0,-999,-999.0,6.6,1008.86,2.0,93.79,3.4,0.1,0.1,eor,2018-11-24
3,433,20181125,10,8.5,2.6,3,0.0,4,0.0,-999,-999.0,7.7,1006.93,4.3,91.88,5.5,2.6,0.6,eor,2018-11-25
4,433,20181126,10,8.5,4.2,3,0.0,0,0.0,-999,-999.0,6.0,1008.51,2.5,82.21,3.9,-1.2,-6.1,eor,2018-11-26


### Rename columns

In [11]:
df_berlin.rename(columns={' TMK': 'Temperature',
                               ' SDK': 'Sun hours',
                               ' RSK': 'Rain',
                               'RSKF': 'Prec. form',
                               '  FM': 'Mean wind velocity [m/s]',
                               '  FX': 'Max wind gust [m/s]',
                               '  NM': 'Cloud coverage'
                              }, inplace=True)

### Data cleaning

In [12]:
interesting_cols = ['Temperature', 'Sun hours', 'Rain', 'Prec. form',
                    'Mean wind velocity [m/s]', 'Max wind gust [m/s]', 'Cloud coverage']

In [13]:
df_berlin[interesting_cols].describe()

,Temperature,Sun hours,Rain,Prec. form,Mean wind velocity [m/s],Max wind gust [m/s],Cloud coverage
count,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000
mean,9.817823,-312.102138,1.504048,2.769173,-359.401454,-354.565525,-156.400970
std,7.877867,466.472348,9.533905,9.182925,482.036198,485.213097,369.080928
min,-18.500000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
25%,3.800000,-999.000000,0.000000,0.000000,-999.000000,-999.000000,1.900000
50%,9.900000,0.400000,0.000000,1.000000,2.600000,7.700000,5.000000
75%,16.100000,5.900000,1.400000,6.000000,4.100000,11.100000,7.000000
max,30.500000,16.300000,119.500000,8.000000,14.500000,36.000000,8.000000


In [14]:
remaining_interesting_cols = ['Temperature', 'Sun hours', 'Rain', 'Prec. form',
                              'Mean wind velocity [m/s]', 'Max wind gust [m/s]']

In [15]:
for col in remaining_interesting_cols:
    display(df_berlin[col].value_counts(dropna=False).head())

5.8     159
6.8     152
5.2     151
5.4     150
16.0    150
Name: Temperature, dtype: int64

-999.0    8358
 0.0      3737
 0.1       346
 0.2       232
 0.3       210
Name: Sun hours, dtype: int64

0.0    14095
0.1     1160
0.2      794
0.3      597
0.4      473
Name: Rain, dtype: int64

0    9153
6    5610
1    5608
4    2817
8    1747
Name: Prec. form, dtype: int64

-999.0    9593
 3.1       490
 3.5       465
 3.4       454
 2.9       450
Name: Mean wind velocity [m/s], dtype: int64

-999.0    9581
 9.8       591
 10.8      558
 7.7       554
 10.3      530
Name: Max wind gust [m/s], dtype: int64

In [16]:
cols_with_missing_data = ['Sun hours', 'Rain', 'Prec. form', 'Mean wind velocity [m/s]', 'Max wind gust [m/s]']

for col in cols_with_missing_data:
    df_berlin[col].replace(-999, 0, inplace=True)

In [17]:
df_berlin[interesting_cols].describe()

,Temperature,Sun hours,Rain,Prec. form,Mean wind velocity [m/s],Max wind gust [m/s],Cloud coverage
count,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000,26483.000000
mean,9.817823,3.180949,1.579493,2.844617,2.468689,6.851950,-156.400970
std,7.877867,4.249427,3.910328,2.920708,2.271665,6.046937,369.080928
min,-18.500000,0.000000,0.000000,0.000000,0.000000,0.000000,-999.000000
25%,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,1.900000
50%,9.900000,0.400000,0.000000,1.000000,2.600000,7.700000,5.000000
75%,16.100000,5.900000,1.400000,6.000000,4.100000,11.100000,7.000000
max,30.500000,16.300000,119.500000,8.000000,14.500000,36.000000,8.000000


# Viz with Altair

In [18]:
df_berlin.head()

,STATIONS_ID,MESS_DATUM,QN_3,Max wind gust [m/s],Mean wind velocity [m/s],QN_4,Rain,Prec. form,Sun hours,SHK_TAG,Cloud coverage,VPM,PM,Temperature,UPM,TXK,TNK,TGK,eor,time
0,433,20181122,10,8.8,4.3,3,0.0,0,0.0,-999,-999.0,5.3,1018.83,1.1,80.79,1.8,0.6,0.3,eor,2018-11-22
1,433,20181123,10,5.8,2.7,3,0.0,4,0.0,-999,-999.0,6.4,1016.12,2.0,90.29,3.4,0.5,0.2,eor,2018-11-23
2,433,20181124,10,4.1,2.1,3,0.1,4,0.0,-999,-999.0,6.6,1008.86,2.0,93.79,3.4,0.1,0.1,eor,2018-11-24
3,433,20181125,10,8.5,2.6,3,0.0,4,0.0,-999,-999.0,7.7,1006.93,4.3,91.88,5.5,2.6,0.6,eor,2018-11-25
4,433,20181126,10,8.5,4.2,3,0.0,0,0.0,-999,-999.0,6.0,1008.51,2.5,82.21,3.9,-1.2,-6.1,eor,2018-11-26


In [19]:
df_berlin.shape

(26483, 20)

In [20]:
df_berlin_all = df_berlin.copy()

In [21]:
df_berlin = df_berlin_all[lambda x: x['time'].dt.year > 2006].copy()
df_berlin.shape

(4933, 20)

In [22]:
base = alt.Chart(df_berlin).encode(alt.X('year(time):O', title='year'))
bar = base.mark_bar().encode(y="sum(Rain):Q").transform_filter(
    {'or': [alt.FieldOneOfPredicate(field='time', timeUnit='month', oneOf=['March']),
            {'and': [alt.FieldOneOfPredicate(field='time', timeUnit='month', oneOf=['April']),
                     alt.FieldLTPredicate(field='time', timeUnit='day', lt=20)]}]}
)
bar.properties(
    width=600,
    title='Sum of Rain between 01.03. and 19.04. per year'
)

alt.Chart(...)

In [23]:
from altair import datum

In [24]:
scale = alt.Scale(
    domain=[0, 0.1, 20],
    range=['gray', 'lightblue', 'darkblue'],
    type='linear'
)

alt.Chart(df_berlin).mark_rect().encode(
    alt.X('monthdate(time):O', title='day'),
    alt.Y('year(time):O', title='month'),
    alt.Color('sum(Rain):Q', scale=scale)
).properties(
    title="Daily sum of rain [mm]",
    width=800,
    height=200
).transform_filter(
    alt.FieldOneOfPredicate(field='time', timeUnit='month', oneOf=['March', 'April'])
)

alt.Chart(...)